# From Decision Trees to Random Forests

```
Authors: Alexandre Gramfort
         Thomas Moreau
```

## Bagging classifiers

We saw that by increasing the depth of the tree, we are going to get an over-fitted model. A way to bypass the choice of a specific depth it to combine several trees together.

Let's start by training several trees on slightly different data. The slightly different dataset could be generated by randomly sampling with replacement. In statistics, this called a boostrap sample. We will use the iris dataset to create such ensemble and ensure that we have some data for training and some left out data for testing.

In [1]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

np.random.seed(42)

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

Before to train several decision trees, we will run a single tree. However, instead to train this tree on `X_train`, we want to train it on a bootstrap sample. You can use the `np.random.choice` function sample with replacement some index. You will need to create a sample_weight vector and pass it to the `fit` method of the `DecisionTreeClassifier`. We provide the `generate_sample_weight` function which will generate the `sample_weight` array.

In [2]:
def bootstrap_idx(X):
    indices = np.random.choice(np.arange(X.shape[0]), size=X.shape[0], replace=True)
    return indices

In [3]:
bootstrap_idx(X_train)

array([ 79,  60,  40,  99,  32,  67,  32,  13,  20,  47,  19,   7,   6,
        66,  16,  32,  47,  75,  58,  85,  21, 109,  29,  37, 101, 108,
        50,  53,   7,  26,  26,  97,  20,  29,  96,  27, 110,  63,  96,
        68,  60,  47,  18,   3,  34,  63,  48,  16,  43,  91,  29,  92,
        45,   5,  98, 104,  36,  23,  92,  45,  52,  94,  98,  59,  96,
        62,  84,  31,  86,  32,  66,  17,  24, 105,  94,  53,  57,  66,
       103,  45,  23,  31,  46,  85,  22,  65,  26, 105,   1,  89,  16,
       103,  32,   8,  42,  47,  38,  92,  41,  25,  98,  49,  24,  23,
        12,  59,   6,  56,  35,  44,  19,  64])

In [4]:
from collections import Counter

Counter(bootstrap_idx(X_train))

Counter({7: 2,
         109: 2,
         15: 5,
         13: 1,
         75: 3,
         86: 2,
         14: 2,
         91: 3,
         97: 1,
         65: 1,
         31: 1,
         62: 4,
         85: 3,
         50: 2,
         24: 1,
         57: 5,
         61: 3,
         21: 2,
         48: 1,
         51: 2,
         41: 1,
         69: 1,
         53: 1,
         106: 1,
         59: 2,
         100: 3,
         96: 1,
         52: 1,
         107: 2,
         4: 1,
         102: 2,
         67: 1,
         5: 1,
         108: 2,
         95: 2,
         93: 2,
         46: 2,
         98: 1,
         54: 2,
         39: 1,
         12: 1,
         105: 1,
         29: 1,
         18: 2,
         16: 1,
         89: 4,
         22: 1,
         8: 1,
         11: 1,
         0: 3,
         110: 1,
         33: 1,
         47: 2,
         88: 1,
         103: 1,
         60: 1,
         63: 1,
         68: 3,
         92: 1,
         66: 1,
         25: 1,
         56: 1,
    

In [5]:
def bootstrap_sample(X, y):
    indices = bootstrap_idx(X)
    return X[indices], y[indices]

In [6]:
X_train_bootstrap, y_train_bootstrap = bootstrap_sample(X_train, y_train)

In [7]:
print(f"Classes distribution in the original data: {Counter(y_train)}")
print(f"Classes distribution in the bootstrap: {Counter(y_train_bootstrap)}")

Classes distribution in the original data: Counter({0: 38, 2: 37, 1: 37})
Classes distribution in the bootstrap: Counter({2: 45, 1: 34, 0: 33})


<div class="alert alert-success">
    <b>EXERCISE: Create a bagging classifier</b>:<br>
    <br>
    A bagging classifier will train several decision tree classifiers, each of them on a different bootstrap sample.
     <ul>
      <li>
      Create several <code>DecisionTreeClassifier</code> and store them in a Python list;
      </li>
      <li>
      Loop over these trees and <code>fit</code> them by generating a bootstrap sample using <code>bootstrap_sample</code> function;
      </li>
      <li>
      To predict with this ensemble of trees on new data (testing set), you can provide the same set to each tree and call the <code>predict</code> method. Aggregate all predictions in a NumPy array;
      </li>
      <li>
      Once the predictions available, you need to provide a single prediction: you can retain the class which was the most predicted which is called a majority vote;
      </li>
      <li>
      Finally, check the accuracy of your model.
      </li>
    </ul>
</div>

In [8]:
from sklearn.tree import DecisionTreeClassifier

list_tree = [DecisionTreeClassifier() for _ in range(3)]
pred_array = []
for tree in list_tree:
    X_train_bootstrap, y_train_bootstrap = bootstrap_sample(X_train, y_train)
    tree.fit(X_train_bootstrap, y_train_bootstrap)

    pred = tree.predict(X_test)
    pred_array.append(pred)

pred_array = np.asarray(pred_array)
print(pred_array.shape)

(3, 38)


In [9]:
from scipy.stats import mode

y_pred = mode(pred_array, axis=0).mode[0]
(y_pred == y_test).mean()

C:\Users\pasqu\AppData\Local\Temp\ipykernel_1216\3987665027.py:3: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_pred = mode(pred_array, axis=0).mode[0]


0.8947368421052632

<div class="alert alert-success">
    <b>EXERCISE: using scikit-learn</b>:
    <br>
    After implementing your own bagging classifier, use a <code>BaggingClassifier</code> from scikit-learn to fit the above data.
</div>

In [10]:
from sklearn.ensemble import BaggingClassifier

clf = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=10,
    random_state=42,
)
clf.fit(X_train, y_train).score(X_test, y_test)

0.9210526315789473

## Random Forests

A very famous classifier is the random forest classifier. It is similar to the bagging classifier. In addition of the bootstrap, the random forest will use a subset of features (selected randomly) to find the best split.

<div class="alert alert-success">
    <b>EXERCISE: Create a random forest classifier</b>:
    <br>
    Use your previous code which was generated several <code>DecisionTreeClassifier</code>. Check the list of the option of this classifier and modify one of the parameters such that only the $\sqrt{F}$ features are used for the splitting. $F$ represents the number of features in the dataset.
</div>

In [11]:
from sklearn.tree import DecisionTreeClassifier

list_tree = [DecisionTreeClassifier(random_state=0) for _ in range(3)]
pred_array = []
for tree in list_tree:
    X_train_bootstrap, y_train_bootstrap = bootstrap_sample(X_train, y_train)
    idx_features = np.random.choice(
        X.shape[1], size=int(np.sqrt(X.shape[1])), replace=False
    )
    X_train_bootstrap = X_train_bootstrap[:, idx_features]
    tree.fit(X_train_bootstrap, y_train_bootstrap)

    pred = tree.predict(X_test[:, idx_features])
    pred_array.append(pred)

pred_array = np.asarray(pred_array)
y_pred = mode(pred_array, axis=0).mode[0]
(y_pred == y_test).mean()

C:\Users\pasqu\AppData\Local\Temp\ipykernel_1216\1419223274.py:15: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_pred = mode(pred_array, axis=0).mode[0]


0.8421052631578947

<div class="alert alert-success">
    <b>EXERCISE: using scikit-learn</b>:
    <br>
    After implementing your own random forest classifier, use a <code>RandomForestClassifier</code> from scikit-learn to fit the above data.
</div>

In [12]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0)
clf.fit(X_train, y_train).score(X_test, y_test)

0.8947368421052632

In [13]:
from figures import plot_forest_interactive

plot_forest_interactive()

interactive(children=(IntSlider(value=0, description='max_depth', max=8), Output()), _dom_classes=('widget-int…